In [ ]:
get-childitem | sort-object

In [ ]:
get-childitem | sort-object -Property Length -Descending

In [ ]:
#filter left, sort right
Measure-Command { Get-Process | Sort-Object -Property Id | Select-Object Name, Id };
#versus
Measure-Command { Get-Process | Select-Object Name, Id | Sort-Object -Property Id };

In [ ]:
Get-Process | Select-Object -Property Name, ID -First 5

In [ ]:
et-Process | Select-Object -Property Name, ID -Last 5

In [ ]:
#use -expandproperty to remove the colum header
Get-Process | Select-Object -ExpandProperty Name -First 5

In [ ]:
Get-Service | Where-Object { $_.Status -eq 'running' -and $_.name -like 'WIN*'}

In [ ]:
Get-ChildItem -path C:\windows\System32 | Group-Object -property extension | sort-object -property count -descending | select-object -first 5

In [ ]:
$ipAddressString = "172.64.0.100"
$ipAddress = [System.Net.IPAddress]::Parse($ipAddressString)

if ($ipAddress.AddressFamily -eq 'InterNetwork' -and (
    ($ipAddress.IPAddressToString -ge [System.Net.IPAddress]::Parse("10.0.0.0").IPAddressToString -and $ipAddress.IPAddressToString -le [System.Net.IPAddress]::Parse("10.255.255.255").IPAddressToString) -or
    ($ipAddress.IPAddressToString -ge [System.Net.IPAddress]::Parse("172.16.0.0").IPAddressToString -and $ipAddress.IPAddressToString -le [System.Net.IPAddress]::Parse("172.31.255.255").IPAddressToString) -or
    ($ipAddress.IPAddressToString -ge [System.Net.IPAddress]::Parse("192.168.0.0").IPAddressToString -and $ipAddress.IPAddressToString -le [System.Net.IPAddress]::Parse("192.168.255.255").IPAddressToString)))
{
    Write-Output "The IP $ipAddressString is in the private IP address space."
} elseif ($ipAddress.AddressFamily -eq 'InterNetwork' -and (
    ($ipAddress.IPAddressToString -ge [System.Net.IPAddress]::Parse("0.0.0.0").IPAddressToString -and $ipAddress.IPAddressToString -le [System.Net.IPAddress]::Parse("9.255.255.255").IPAddressToString) -or
    ($ipAddress.IPAddressToString -ge [System.Net.IPAddress]::Parse("11.0.0.0").IPAddressToString -and $ipAddress.IPAddressToString -le [System.Net.IPAddress]::Parse("172.15.255.255").IPAddressToString) -or
    ($ipAddress.IPAddressToString -ge [System.Net.IPAddress]::Parse("172.32.0.0").IPAddressToString -and $ipAddress.IPAddressToString -le [System.Net.IPAddress]::Parse("192.167.255.255").IPAddressToString) -or
    ($ipAddress.IPAddressToString -ge [System.Net.IPAddress]::Parse("192.169.0.0").IPAddressToString -and $ipAddress.IPAddressToString -le [System.Net.IPAddress]::Parse("255.255.255.255").IPAddressToString)))
{
    Write-Output "The IP $ipAddressString is in the public IP address space."
} else {
    Write-Output "$ipAddressString is an Invalid IP address."
}

In [ ]:
PS C:\> $FreeSpace = (Get-PSDrive -PSProvider FileSystem | Select-Object -Property @{Name='GBFree'; Expression={[math]::Round($_.Free / 1GB, 2)}} -First 1).GBFree
PS C:\> if($FreeSpace -gt 250) {
            Write-Host "You have more than 250GB of disk space available"
        } else {
            Write-Host "You have less than 250GB of disk space available"
        }

In [ ]:
PS C:\> $FreeSpace = .5
PS C:\> if($FreeSpace -gt 250) {
            Write-Host "You have more than 250GB of disk space available"
        } elseif (($FreeSpace -gt 100) -and ($FreeSpace -lt 250)){
            Write-Host "You have more than 100GB of disk space, but less then 250GB available"
        } elseif (($FreeSpace -lt 100) -and ($FreeSpace -gt 1)){
            Write-Host "You have less than 100GB of disk space available"
        } elseif ($FreeSpace -lt 1) {
            Write-Host "Get a new hard drive!"
        } else {Write-Host "Monitoring Microsoft Edge process every 60 seconds." `n
        $FilePath = (Get-CimInstance Win32_Process | Select-Object -Property Name, ProcessID, ParentProcessID, Path | Where-Object {$_.Name -eq "msedge.exe"}).Path
        $processToMonitor = "msedge"
        do {
            $isProcessRunning = Get-Process -Name $processToMonitor

            if ($isProcessRunning) {
                Write-Host "Warning: $processToMonitor is running, performing analysis on the process." `n
                Write-Host "-------------Gathering Process Information...--------------"
                Start-Sleep -Seconds 3
                Write-Output (Get-CimInstance Win32_Process | Select-Object -Property Name, ProcessID, ParentProcessID, Path | Where-Object {$_.Name -eq "msedge.exe"} | Format-Table)

                Write-Host "-------------Gathering Process Filehash...--------------"
                Start-Sleep -Seconds 3
                $FileHash = (Get-FileHash -Algorithm SHA256 -Path $FilePath | Format-Table)
                Write-Output $FileHash

                Write-Host "-------------Gathering Network Information...--------------"
                Start-Sleep -Seconds 3
                $ProcessIDs = (Get-Process | Select-Object -Property Name, ID | Where-Object {$_.Name -eq "msedge"}).Id
                $Network = foreach ($ID in $ProcessIDs) {
                    Get-NetTCPConnection | Select-Object -Property LocalAddress, RemoteAddress, RemotePort, OwningProcess, State | Where-Object { $_.OwningProcess -eq $ID -and $_.State -ne "Bound" }
                }
                Write-Output $Network | Format-Table
            }
            Start-Sleep -Seconds 60
        } until (-not $isProcessRunning)
            Write-Host "You have less than 250GB of disk space available"
        }

In [ ]:
PS C:\> Switch (3) {
    1 { Write-Host "You selected menu item 1" }
    2 { Write-Host "You selected menu item 2" }
    3 { Write-Host "You selected menu item 3" }
    Default { Write-Host "You did not select a valid option" }
}


In [ ]:
PS C:\> $ip = '172.64.0.100'
PS C:\> Switch -WildCard ($ip) {
            "192.168.*" { Write-Host "This computer is on the internal local area network" }
            "10.15.*" { Write-Host "This computer is in the Branch network" }
            "172.64.*" { Write-Host "This computer is in the DMZ network" }
            Default { Write-Host "This computer is not on the network" }
        }

In [ ]:
PS C:\> $ipArray = @('172.64.0.100', '10.50.35.169','10.15.0.100','22.25.55.255')
PS C:\> $ipArray | ForEach-Object {
            $ip = $_
            $index = $ipArray.IndexOf($ip)

            switch -Wildcard ($ip) {
                "192.168.*" { Write-Host ("$ip is on the internal local area network") }
                "10.15.*" { Write-Host ("$ip is in the Branch network") }
                "172.64.*" { Write-Host ("$ip is in the DMZ network") }
                Default { Write-Host ("$ip is not on the network") }
            }
        }


In [ ]:
taskkill /F /IM msedge.exe


In [ ]:
Write-Host "Monitoring Microsoft Edge process every 60 seconds." `n
        $FilePath = (Get-CimInstance Win32_Process | Select-Object -Property Name, ProcessID, ParentProcessID, Path | Where-Object {$_.Name -eq "msedge.exe"}).Path
        $processToMonitor = "msedge"
        do {
            $isProcessRunning = Get-Process -Name $processToMonitor

            if ($isProcessRunning) {
                Write-Host "Warning: $processToMonitor is running, performing analysis on the process." `n
                Write-Host "-------------Gathering Process Information...--------------"
                Start-Sleep -Seconds 3
                Write-Output (Get-CimInstance Win32_Process | Select-Object -Property Name, ProcessID, ParentProcessID, Path | Where-Object {$_.Name -eq "msedge.exe"} | Format-Table)

                Write-Host "-------------Gathering Process Filehash...--------------"
                Start-Sleep -Seconds 3
                $FileHash = (Get-FileHash -Algorithm SHA256 -Path $FilePath | Format-Table)
                Write-Output $FileHash

                Write-Host "-------------Gathering Network Information...--------------"
                Start-Sleep -Seconds 3
                $ProcessIDs = (Get-Process | Select-Object -Property Name, ID | Where-Object {$_.Name -eq "msedge"}).Id
                $Network = foreach ($ID in $ProcessIDs) {
                    Get-NetTCPConnection | Select-Object -Property LocalAddress, RemoteAddress, RemotePort, OwningProcess, State | Where-Object { $_.OwningProcess -eq $ID -and $_.State -ne "Bound" }
                }
                Write-Output $Network | Format-Table
            }
            Start-Sleep -Seconds 60
        } until (-not $isProcessRunning)

In [ ]:
Get-CIMInstance Win32_networkadapterconfiguration | select-object -property dnshostname, interfaceindex, ipaddress, ipsubnet | format-list

In [ ]:
get-CIMInstance -Classname Win32_Process | select-object name, processid, partentprocessid

In [ ]:
# Enable PSRemoting and skip the network profile check to avoid errors
Enable-PSRemoting -SkipNetworkProfileCheck -Force
# Check trusted hosts file
Get-Item WSMan:\localhost\client\TrustedHosts
# Set trusted hosts to allow your connection to a desired host or hosts. Multiple trusted hosts can be created in a comma separated list and stored in a variable for use with this command
# Enter "Y" when prompted
Set-Item WSMan:\localhost\Client\TrustedHosts -Value 10.50.35.169
# Will allow incoming and outgoing - risky to put a *
$creds = get-credential
enter-pssession -computername = 10.50.35.169 -credential $creds

In [ ]:
invoke-command -computername 192.168.1.1 -credential $creds -scriptblock{get-process}

In [ ]:
$network = invoke-command -computername 192.168.1.250 -credential $creds -scriptblock{get-nettcpconnection}; $network | out-file -filepath "C:\Users\Student\Desktop\NetworkResults.txt"